In [1]:
from dataclasses import dataclass
import sample
import json
import pandas as pd
from solver import perform_one_token_cpc, perform_cot_cpc
from solver import perform_one_token_cpc, perform_cot_cpc
from judge import JudgeResult

When the ai considers whether to step back or not, does its one-word answer differ from its CoT answer?

In [2]:
@dataclass
class Result:
    context: sample.Context
    one_token_cpc_result: str
    cot_cpc_result: str

from dotenv import load_dotenv
load_dotenv()

from llm import LLM
from openai import OpenAI
llm = LLM(OpenAI(), "gpt-3.5-turbo")

Each 'passage' is a lengthy text where we are reasoning through a problem.
Consider progressively larger context parts of each passage (that is, we are 'checking in' as we proceed through reading the passage), and for each context part, ask the LLM if the current approach is working or not.

In [3]:
passages = json.load(open("data/passages1.json"))
experiment1 = pd.DataFrame(columns=["category", "context", "cot_cpc_thoughts", "one_token_cpc_result", "cot_cpc_result"])

In [4]:
for context in sample.checkpoints(passages["coding_interviews"][0], 1000):
    one_token_cpc_result = perform_one_token_cpc(llm, context)
    (cot_cpc_thoughts, cot_cpc_result) = perform_cot_cpc(llm, context)
    print(context.text)
    print(one_token_cpc_result)
    print(cot_cpc_thoughts)
    print(cot_cpc_result)
    print()


BadRequestError: Error code: 400 - {'error': {'message': "Invalid key in 'logit_bias': Yes. You should only be submitting non-negative integers.", 'type': 'invalid_request_error', 'param': 'logit_bias', 'code': None}}

In [5]:
for category, passages_list in passages.items():
    for passage_index, passage in enumerate(passages_list):
        print(f"Category {category}: passage {passage_index} of {len(passages_list)}...")
        checkpoints = (text for document in passages for text in sample.checkpoints(passage, 1000))
        for context in checkpoints:
            one_token_cpc_result = perform_one_token_cpc(llm, context)
            (cot_cpc_thoughts, cot_cpc_result) = perform_cot_cpc(llm, context)
            experiment1 = pd.concat([experiment1, pd.DataFrame(
                {"category": category, "context": context.text, "one_token_cpc_result": one_token_cpc_result,
                 "cot_cpc_thoughts": cot_cpc_thoughts, "cot_cpc_result": cot_cpc_result}, index=[0])],
                                    ignore_index=True)


Category proofs: passage 0 of 10...
Category proofs: passage 1 of 10...
Category proofs: passage 2 of 10...
Category proofs: passage 3 of 10...
Category proofs: passage 4 of 10...
Category proofs: passage 5 of 10...
Category proofs: passage 6 of 10...
Category proofs: passage 7 of 10...
Category proofs: passage 8 of 10...
Category proofs: passage 9 of 10...
Category coding_interviews: passage 0 of 10...
Category coding_interviews: passage 1 of 10...
Category coding_interviews: passage 2 of 10...
Category coding_interviews: passage 3 of 10...
Category coding_interviews: passage 4 of 10...
Category coding_interviews: passage 5 of 10...
Category coding_interviews: passage 6 of 10...
Category coding_interviews: passage 7 of 10...
Category coding_interviews: passage 8 of 10...
Category coding_interviews: passage 9 of 10...


For each result, determine whether the result is good (the two cpc methods agreed) or bad (they disagreed).

In [10]:
experiment1["score"] = experiment1.apply(
    lambda row: 1.0 if row.one_token_cpc_result.lower() == row.cot_cpc_result.lower() else 0.0, axis=1)

In [11]:
experiment1.to_csv("experiment1.csv")

In [8]:
# Print the results
print(experiment1)

              category                                            context  \
0               proofs  For this problem, I need to prove that for any...   
1               proofs  For this problem, I need to prove that for any...   
2               proofs  For this problem, I need to prove that for any...   
3               proofs  For this problem, I need to prove that for any...   
4               proofs  For this problem, I need to prove that for any...   
..                 ...                                                ...   
149  coding_interviews  For this coding interview question, I need to ...   
150  coding_interviews  For this coding interview question, I need to ...   
151  coding_interviews  For this coding interview question, I need to ...   
152  coding_interviews  For this coding interview question, I need to ...   
153  coding_interviews  For this coding interview question, I need to ...   

                                      cot_cpc_thoughts one_token_cpc_result